데이터 : https://www.kaggle.com/rodolfomendes/abalone-dataset  
원본출처 : https://machinelearningmastery.com/columntransformer-for-numerical-and-categorical-data/

In [1]:
import pandas as pd

In [2]:
from pandas import read_csv
# load dataset
url = 'https://raw.githubusercontent.com/jbrownlee/Datasets/master/abalone.csv'
dataframe = read_csv(url, header=None)

In [3]:
dataframe.set_axis(["Sex", "Length", "Diameter", "Height", "Whole weight", "Shucked weight", "Viscera weight", "Shell weight", "Rings"],
                    axis=1, inplace=True)
dataframe.head(3)

,Sex,Length,Diameter,Height,Whole weight,Shucked weight,Viscera weight,Shell weight,Rings
0,M,0.455,0.365,0.095,0.5140,0.2245,0.1010,0.15,15
1,M,0.350,0.265,0.090,0.2255,0.0995,0.0485,0.07,7
2,F,0.530,0.420,0.135,0.6770,0.2565,0.1415,0.21,9


In [4]:
X, y = dataframe.drop("Rings", axis=1), dataframe["Rings"]
print(X.shape, y.shape)

(4177, 8) (4177,)


전복의 진주 갯수로 전복의 나이를 알 수 있다고 한다. 따라서 본 dataset의 목적은 Rings를 예측하는 것이라고 할 수 있다.

In [5]:
numerical_ix = X.select_dtypes(include=['int64', 'float64']).columns
categorical_ix = X.select_dtypes(include=['object', 'bool']).columns

In [6]:
numerical_ix

Index(['Length', 'Diameter', 'Height', 'Whole weight', 'Shucked weight',
       'Viscera weight', 'Shell weight'],
      dtype='object')

In [7]:
categorical_ix

Index(['Sex'], dtype='object')

***

In [8]:
from sklearn.preprocessing import OneHotEncoder, MinMaxScaler
from sklearn.compose import ColumnTransformer

In [9]:
t = [('cat', OneHotEncoder(), categorical_ix), ('num', MinMaxScaler(), numerical_ix)]
col_transform = ColumnTransformer(transformers=t)

In [19]:
col_transform

ColumnTransformer(n_jobs=None, remainder='drop', sparse_threshold=0.3,
                  transformer_weights=None,
                  transformers=[('cat',
                                 OneHotEncoder(categorical_features=None,
                                               categories=None, drop=None,
                                               dtype=<class 'numpy.float64'>,
                                               handle_unknown='error',
                                               n_values=None, sparse=True),
                                 Index(['Sex'], dtype='object')),
                                ('num',
                                 MinMaxScaler(copy=True, feature_range=(0, 1)),
                                 Index(['Length', 'Diameter', 'Height', 'Whole weight', 'Shucked weight',
       'Viscera weight', 'Shell weight'],
      dtype='object'))],
                  verbose=False)

In [15]:
from sklearn.svm import SVR
from sklearn.pipeline import Pipeline

In [16]:
model = SVR(kernel='rbf',gamma='scale',C=100)
pipeline = Pipeline(steps=[('prep',col_transform), ('m', model)])

In [17]:
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from numpy import absolute, mean, std

In [18]:
cv = KFold(n_splits=10, shuffle=True, random_state=1)
# evaluate the pipeline using cross validation and calculate MAE
scores = cross_val_score(pipeline, X, y, scoring='neg_mean_absolute_error', cv=cv, n_jobs=-1)
# convert MAE scores to positive values
scores = absolute(scores)
# summarize the model performance
print('MAE: %.3f (%.3f)' % (mean(scores), std(scores)))

MAE: 1.465 (0.047)
